In [ ]:
# 실행하고 링크 누르기
# 런타임 유형 v2-8 TPU

In [1]:
import os
print("Setting up GCS access...")
from google.colab import auth
os.environ['USE_AUTH_EPHEM'] = '0'
auth.authenticate_user()

Setting up GCS access...


# Set Up

In [2]:
#버전은 언제든지 변경이 될 수 있음. 라이브러리 문제. 아래 cell에서 import 되지 않으면 맞는 버전 찾기
from IPython.display import clear_output
!pip install gcsfs
!pip install https://storage.googleapis.com/cloud-tpu-tpuvm-artifacts/tensorflow/tf-2.15.0/tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install tensorflow
!pip install t5
#!pip install 'jax[tpu]==0.4.26' -f https://storage.googleapis.com/jax-releases/libtpu_releases.html -f https://storage.googleapis.com/jax-releases/jax_releases.html
#!pip install jaxlib==0.4.26
!pip install tensorflow-gcs-config==2.15.0
!pip install -q tensorflow-text==2.15.0
clear_output()
print("Installing dependencies...")

Installing dependencies...


In [3]:
import functools
import os
import gin
from contextlib import contextmanager
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import logging as py_logging
#tf.app.flags.DEFINE_string ('f', '', '')

import t5

In [4]:
import tensorflow_gcs_config
import tensorflow.compat.v1 as tf
TPU_TOPOLOGY = "2x2"


try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')  # TPU detection
    tf.config.experimental_connect_to_cluster(resolver)
    tf.config.experimental.list_logical_devices('TPU')
    tf.config.experimental.list_physical_devices('TPU')
    print("All devices: ", tf.config.list_logical_devices('TPU'))
    s = tf.Session()
    print(s.list_devices())
    #with tf.device():
    tensorflow_gcs_config.configure_gcs_from_colab_auth(tf.config.list_logical_devices('TPU')[2])
except ValueError:
    raise BaseException(
        'ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!'
    )

tf.disable_v2_behavior()

#LOGGING
tf.get_logger().propagate = False
py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Instructions for updating:
non-resource variables are not supported in the long term


All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]
[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, -1561952162512122479), _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 17179869184, -8215740505360834502), _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 17179869184, -894499189513088272), _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8819508387627147617), _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 17179869184, -1827041777991735787), _DeviceAttribut

# Variables

In [5]:
task = "masking" #@param ['masking']
scheduler = "noam" #@param ['noam']
VOCAB_PREFIX = 'tokenizer' #@param {type: "string"}
TOKENIZER_DIR = f"gs://log_gen/tokenizer/tokens/"
path_pretraining_task1 = f"gs://log_gen/pretraining/{task}/pretraining.tsv" #기존 LEONID 데이터셋
path_word_pretraining_task = f"gs://log_gen/pretraining/custom/word/no_dup_masked_methods.tsv" # word 기준으로 나눈 데이터셋
path_token_pretraining_task = f"gs://log_gen/pretraining/custom/token/no_dup_masked_methods_with_token.tsv" # token 기준으로 나눈 데이터셋
# Storage paths
PRETRAIN_MODEL_DIR = f"gs://log_gen/pretrained-model/{task}" # LEONID 데이터셋의 모델 위치
PRETRAIN_MODEL_TOKEN_DIR = f"gs://log_gen/pretrained-model/custom/token_real" #token 기준으로 나눈 데이터셋의 모델 위치
PRETRAIN_MODEL_WORD_DIR= f"gs://log_gen/pretrained-model/custom/word_real" #word 기준으로 나눈 데이터셋의 모델 위치

#PRETRAIN_MODEL_DIR - path_pretraining_task1 쌍
#PRETRAIN_MODEL_TOKEN_DIR - path_token_pretraining_task 쌍
#PRETRAIN_MODEL_WORD_DIR - path_word_pretraining_task 쌍

# Vocab

In [6]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


vocab_model_path = os.path.join(TOKENIZER_DIR, f'{VOCAB_PREFIX}.model')
vocab_path = os.path.join(TOKENIZER_DIR, f'{VOCAB_PREFIX}.vocab')


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask

num_special_mask_tokens = 100 #@param {type: "integer"}

def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, num_special_mask_tokens)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

In [7]:
#Skip this cell for running the pre-training on the second task only
nq_tsv_path = {
    "train":      #path_pretraining_task1, #12671475
                  #path_word_pretraining_task,#21820261
                  path_token_pretraining_task #21923714
} # 여기서 데이터셋 파일 위치 결정

num_nq_examples_task1 = dict(train=21923714) # 데이터셋 파일에 따라 개수 바꾸기, 위에 있는 train에 파일에 따라 개수 적어놓음

def load_dataset(split, shuffle_files=True):
  del shuffle_files

  ds = tf.data.TextLineDataset(nq_tsv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)

  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

## Examples

In [8]:
print("A few raw train examples...")
for ex in tfds.as_numpy(load_dataset("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'"public static List<<extra_id_0> netSocketAddress> getAddresses(String s<extra_id_1> { <extra_id_2> (s == null) { throw new NullPoint<extra_id_3> Exception(""Null host list""); } if (s<extra_id_4><extra_id_5><extra_id_6> ().<extra_id_7> qual<extra_id_8> (<extra_id_9> "")) { throw new Il<extra_id_10> Arg<extra_id_11><extra_id_12> Exception(<extra_id_13><extra_id_14> hosts in list: ``<extra_id_15><extra_id_16> s<extra_id_17> ""\'\'""); } ArrayList<InetSocketAddress> addrs = new ArrayList<InetSo<extra_id_18><extra_id_19> Address>();<extra_id_20> (String hoststuff <extra_id_21> <extra_id_22> .split(""<extra_id_23> ?:\\\\s|,)+"")<extra_id_24> { if (hosts<extra_id_25> uff.equals<extra_id_26> """"))<extra_id_27> { continue; } int<extra_id_28> Colon = hoststuff.lastIndex<extra_id_29><extra_id_30> (\'<extra_id_31> \'); if (<extra_id_32> Colon < 1) { throw new Illegal<extra_id_33> umentException(""In<extra_id_34> server ``"" +<extra_id_35> tuff<extra_id_36

In [9]:
print("A few raw train examples...")
for ex in tfds.as_numpy(load_dataset("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'"public static List<<extra_id_0> netSocketAddress> getAddresses(String s<extra_id_1> { <extra_id_2> (s == null) { throw new NullPoint<extra_id_3> Exception(""Null host list""); } if (s<extra_id_4><extra_id_5><extra_id_6> ().<extra_id_7> qual<extra_id_8> (<extra_id_9> "")) { throw new Il<extra_id_10> Arg<extra_id_11><extra_id_12> Exception(<extra_id_13><extra_id_14> hosts in list: ``<extra_id_15><extra_id_16> s<extra_id_17> ""\'\'""); } ArrayList<InetSocketAddress> addrs = new ArrayList<InetSo<extra_id_18><extra_id_19> Address>();<extra_id_20> (String hoststuff <extra_id_21> <extra_id_22> .split(""<extra_id_23> ?:\\\\s|,)+"")<extra_id_24> { if (hosts<extra_id_25> uff.equals<extra_id_26> """"))<extra_id_27> { continue; } int<extra_id_28> Colon = hoststuff.lastIndex<extra_id_29><extra_id_30> (\'<extra_id_31> \'); if (<extra_id_32> Colon < 1) { throw new Illegal<extra_id_33> umentException(""In<extra_id_34> server ``"" +<extra_id_35> tuff<extra_id_36

# Dataset Preprocessing

In [10]:
def preprocessing(ds):

  def to_inputs_and_targets(ex):
        inputs = tf.strings.join(['MASKING: ' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }

  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

## Examples

In [11]:
print("A few preprocessed train examples...")
sample = tfds.as_numpy(preprocessing(load_dataset("train").take(5)))
for ex in sample:
  print(ex)

A few preprocessed train examples...
{'inputs': b'MASKING: "public static List<<extra_id_0> netSocketAddress> getAddresses(String s<extra_id_1> { <extra_id_2> (s == null) { throw new NullPoint<extra_id_3> Exception(""Null host list""); } if (s<extra_id_4><extra_id_5><extra_id_6> ().<extra_id_7> qual<extra_id_8> (<extra_id_9> "")) { throw new Il<extra_id_10> Arg<extra_id_11><extra_id_12> Exception(<extra_id_13><extra_id_14> hosts in list: ``<extra_id_15><extra_id_16> s<extra_id_17> ""\'\'""); } ArrayList<InetSocketAddress> addrs = new ArrayList<InetSo<extra_id_18><extra_id_19> Address>();<extra_id_20> (String hoststuff <extra_id_21> <extra_id_22> .split(""<extra_id_23> ?:\\\\s|,)+"")<extra_id_24> { if (hosts<extra_id_25> uff.equals<extra_id_26> """"))<extra_id_27> { continue; } int<extra_id_28> Colon = hoststuff.lastIndex<extra_id_29><extra_id_30> (\'<extra_id_31> \'); if (<extra_id_32> Colon < 1) { throw new Illegal<extra_id_33> umentException(""In<extra_id_34> server ``"" +<extra_id_3

In [12]:
print("A few preprocessed train examples...")
sample = tfds.as_numpy(preprocessing(load_dataset("train").take(5)))
for ex in sample:
  print(ex)

A few preprocessed train examples...
{'inputs': b'MASKING: "public static List<<extra_id_0> netSocketAddress> getAddresses(String s<extra_id_1> { <extra_id_2> (s == null) { throw new NullPoint<extra_id_3> Exception(""Null host list""); } if (s<extra_id_4><extra_id_5><extra_id_6> ().<extra_id_7> qual<extra_id_8> (<extra_id_9> "")) { throw new Il<extra_id_10> Arg<extra_id_11><extra_id_12> Exception(<extra_id_13><extra_id_14> hosts in list: ``<extra_id_15><extra_id_16> s<extra_id_17> ""\'\'""); } ArrayList<InetSocketAddress> addrs = new ArrayList<InetSo<extra_id_18><extra_id_19> Address>();<extra_id_20> (String hoststuff <extra_id_21> <extra_id_22> .split(""<extra_id_23> ?:\\\\s|,)+"")<extra_id_24> { if (hosts<extra_id_25> uff.equals<extra_id_26> """"))<extra_id_27> { continue; } int<extra_id_28> Colon = hoststuff.lastIndex<extra_id_29><extra_id_30> (\'<extra_id_31> \'); if (<extra_id_32> Colon < 1) { throw new Illegal<extra_id_33> umentException(""In<extra_id_34> server ``"" +<extra_id_3

# Set up task

In [13]:
t5.data.TaskRegistry.remove('masking')
t5.data.TaskRegistry.add(
    "masking",
    dataset_fn=load_dataset,
    splits=["train"],
    text_preprocessor=preprocessing,
    output_features = DEFAULT_OUTPUT_FEATURES,
    num_input_examples=num_nq_examples_task1
)

## Mixture

In [14]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("pretraining")
t5.data.MixtureRegistry.add(
    "pretraining",
    ["masking"],
    default_rate=_rate_num_input_examples
)

## Model

In [18]:
"""AttributeError                            Traceback (most recent call last)
<ipython-input-16-235c7bc08e88> in <cell line: 1>()
----> 1 from mesh_tensorflow.transformer.learning_rate_schedules import learning_rate_schedule_noam
      2 from t5 import models
      3 # Model properties
      4 MODEL_SIZE = "small"
      5 model_parallelism, train_batch_size, keep_checkpoint_max = {

2 frames
/usr/local/lib/python3.10/dist-packages/mesh_tensorflow/tpu_variables.py in <module>
    224
    225
--> 226 ops.register_tensor_conversion_function(ReplicatedVariable, _tensor_conversion)
    227
    228 if not TF_23:

AttributeError: module 'tensorflow.python.framework.ops' has no attribute 'register_tensor_conversion_function'"""
#위와 같은 에러가 아래 cell에서 발생 가능

# 에러에서 "/usr/local/lib/python3.10/dist-packages/mesh_tensorflow/tpu_variables.py/usr/local/lib/python3.10/dist-packages/mesh_tensorflow/tpu_variables.py"를 누르면 tpu_variable.py가 옆에 나옴
# tpu_variable.py 226줄에서 "ops.register_tensor_conversion_function(ReplicatedVariable, _tensor_conversion)" 를
# ops.tensor_conversion_registry.register_tensor_conversion_function(ReplicatedVariable, _tensor_conversion) 로 변경

'AttributeError                            Traceback (most recent call last)\n<ipython-input-16-235c7bc08e88> in <cell line: 1>()\n----> 1 from mesh_tensorflow.transformer.learning_rate_schedules import learning_rate_schedule_noam\n      2 from t5 import models\n      3 # Model properties\n      4 MODEL_SIZE = "small"\n      5 model_parallelism, train_batch_size, keep_checkpoint_max = {\n\n2 frames\n/usr/local/lib/python3.10/dist-packages/mesh_tensorflow/tpu_variables.py in <module>\n    224 \n    225 \n--> 226 ops.register_tensor_conversion_function(ReplicatedVariable, _tensor_conversion)\n    227 \n    228 if not TF_23:\n\nAttributeError: module \'tensorflow.python.framework.ops\' has no attribute \'register_tensor_conversion_function\''

In [19]:
from mesh_tensorflow.transformer.learning_rate_schedules import learning_rate_schedule_noam
from t5 import models
# Model properties
MODEL_SIZE = "small"
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 128, 16), #change batch 128|
    "base": (2, 16, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(PRETRAIN_MODEL_TOKEN_DIR)#PRETRAIN_MODEL_WORD_DIR)#PRETRAIN_MODEL_DIR)

# Mesh Tensorflow Transformer
model = models.mtf_model.MtfModel(
    model_dir=PRETRAIN_MODEL_TOKEN_DIR, #PRETRAIN_MODEL_WORD_DIR,#PRETRAIN_MODEL_DIR,
    tpu='local',
    #tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = learning_rate_schedule_noam,
    sequence_length={"inputs": 512, "targets": 512},
    save_checkpoints_steps=10000,
    keep_checkpoint_max=keep_checkpoint_max,
    iterations_per_loop=100,
)

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set up learning Scheduler

In [21]:
remote_gin_path = f"gs://log_gen/learning_rate_scheduler/{scheduler}/operative_config.gin"
local_gin_path = "/content/operative_config.gin"
!gsutil -m cp $remote_gin_path $local_gin_path

Copying gs://log_gen/learning_rate_scheduler/noam/operative_config.gin...
/ [1/1 files][ 12.4 KiB/ 12.4 KiB] 100% Done                                    
Operation completed over 1 objects/12.4 KiB.                                     


In [22]:
with gin.unlock_config():
    gin.parse_config_file(local_gin_path)

# Pre-train model

In [23]:
#체크 포인트가 있으면 로드
checkpoint_dir = PRETRAIN_MODEL_TOKEN_DIR
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    print(f"Resuming from checkpoint: {latest_checkpoint}")
else:
    print("No checkpoint found. Starting training from scratch.")

Resuming from checkpoint: gs://log_gen/pretrained-model/custom/token_real/model.ckpt-500000


In [ ]:
#checkpoint 부터 다시 학습
TRAIN_STEPS = 500000 #@param {type: "integer"}
model.train("pretraining", TRAIN_STEPS, init_checkpoint=latest_checkpoint) #pretraining = mixture name #checkpoint가 있다면
#model.train("pretraining", TRAIN_STEPS) #checkpoint가 없다면

From /usr/local/lib/python3.10/dist-packages/mesh_tensorflow/transformer/utils.py:2043: TPUConfig.__new__ (from tensorflow_estimator.python.estimator.tpu.tpu_config) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras instead.
From /usr/local/lib/python3.10/dist-packages/mesh_tensorflow/transformer/utils.py:2059: RunConfig.__init__ (from tensorflow_estimator.python.estimator.tpu.tpu_config) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras instead.
From /usr/local/lib/python3.10/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_config.py:268: RunConfig.__init__ (from tensorflow_estimator.python.estimator.run_config) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras instead.
From /usr/local/lib/python3.10/dist-packages/mesh_tensorflow/transformer/utils.py:2096: TPUEstimator.__init__ (from tensorflow_estimator.python.estimator.tpu.tpu_estimator) i